Code cleanup. Upon analyzing the code, it looks like
1. repeated sentences
2. There's some shit going on with emojis
3. There are some words concatenated together, especially the last word of a sentence and the first word of the next sentence.
4. There are some sound effects, like "[Music]". 
We need to get rid of these things.
Data cleaning.

1. Find all files in all 'txt' directories.

In [3]:
import glob

In [45]:

def generateListOfWords(filepath):
    print(filepath)
    listOfWords=[]
    with open(filepath, 'r') as f:
        lines = f.readlines()
        strippedLines = [line.strip() for line in lines if line.strip()]
        for i in range (0, len(strippedLines)-1, 2)[:4]:
            if (strippedLines[i+1] == strippedLines[i+1]):
                strippedLines.remove(strippedLines[i])
        listOfWords = [strippedLine.split() for strippedLine in strippedLines]
        print(listOfWords)
        #1. separate words like coming.I: find all quotation, punctuation and split and remove the target
        #2. separate words like helloBut. idk how to handle this tbh

for filepath in glob.glob("*/*/*.txt")[1:2]:
    generateListOfWords(filepath)


UCY6Ij8zOds0WJEeqCLOnqOQ/txt/O4IxMlqY3hI.txt
< that there's kind of like a trend to thethat there's kind of like a trend to the
> that there's kind of like a trend to the
< TV shows and movies that I talk about
> you know I mean like if there's anyou know I mean like if there's an
< average ly boring teenage girl fallingaverage ly boring teenage girl falling
> average ly boring teenage girl falling
< in love with like a half vampire half
> Teletubby or whatever like you betterTeletubby or whatever like you better
["that there's kind of like a trend to the", 'TV shows and movies that I talk aboutTV shows and movies that I talk about', "you know I mean like if there's anyou know I mean like if there's an", "you know I mean like if there's an", 'average ly boring teenage girl falling', 'in love with like a half vampire halfin love with like a half vampire half', 'Teletubby or whatever like you betterTeletubby or whatever like you better', 'Teletubby or whatever like you better', "believe 